仅仅使用ufunc并不能够进行复杂的并行操作，因此需要使用CUDA Kernel函数进行计算的并行化。
在这里，需要对CUDA C并行计算方法有基本的认识。
计算中需要将任务空间分配给多个`grid`共同完成。一个`grid`由多个`block`组成，每个`block`中由多个`thread`组成。
`thread`运行在一个核心上，`block`运行在Streaming Multiprocessor(SM)上，`grid`运行在整个GPU卡上
这种任务分配方式遵循以下规则：
+ `block`中的`thread`数量应该是32的倍数，且数量位于**128~512**之间
+ `grid`中的`block`数量应确保尽可能地利用整个GPU。在启动计算阶段，`block`的数量是GPU上**多处理器数量的2-4倍**，通常在**20~100**之间。
+ CUDA内核的启动开销确实取决于块的数量，所以当输入规模非常大时，最好不要启动线程数量等于输入元素数量的网格。

In [17]:
from numba import cuda


@cuda.jit
def add_kernel(x, y, out):
    block_size = cuda.blockDim.x  # 每个block中的thread数量
    grid_size = cuda.gridDim.x  # 每个grid中的block数量

    tx = cuda.threadIdx.x  # block中thread的索引，编号从0 ~ blockDim.x-1
    ty = cuda.blockIdx.x  # gird中block的索引，编号从0 ~ gridDim.x-1

    start = tx + ty * block_size  # thread_ID + block_ID * block_size
    stride = block_size * grid_size  # 整个grid有多少thread

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

In [20]:
import cupy as cp

n = 1_000_000
x = cp.arange(n).astype(cp.float32)
y = 2 * x
out = cp.empty_like(x)

threads_per_block = 128
blocks_per_grid = 30

% timeit -r 1 -n 5 add_kernel[blocks_per_grid, threads_per_block](x, y, out)

1.88 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


上面两个cell可以写得更简洁一些，如下所示

In [30]:
from numba import cuda


@cuda.jit
def add_kernel(x, y, out):
    start = cuda.grid(1)  # 一维array空间的thread起点
    stride = cuda.gridsize(1)  # 每个grid包含的thread数量

    # 下面循环的含义是
    # i代表了每个grid计算任务的thread ID
    # 给定了每个grid起始的thread ID后，该grid中每个thread做着循环体中相同的工作
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]


@cuda.jit
def add_kernel_thread(x, y, out):
    threadID = cuda.grid(1)  # 对于一维array，grid中thread的编号
    if threadID < x.shape[0]:  # 对于在1d array长度范围内的数组的元素进行操作
        out[threadID] = x[threadID] + y[threadID]

In [39]:
import cupy as cp

n = 10_000_000

x = cp.arange(n).astype(cp.float32)
y = x * 2
out = cp.empty_like(x)

threads_per_block = 128
blocks_per_grid = 30


% timeit -r 1 -n 1 add_kernel[blocks_per_grid, threads_per_block](x, y, out)
# 由于host在调用CUDA kernel函数后，cpu和gpu便异步计算.
# 调用`cuda.synchronize()`便能使host等待kernel计算完成后再进行下步计算。
# 因此一般在调用kernel函数的下一行加入此代码。
cuda.synchronize()

% timeit -r 1 -n 1 add_kernel_thread[blocks_per_grid, threads_per_block](x, y, out)
cuda.synchronize()

16 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.05 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


接下来测试一个三维array的相加操作

In [1]:
from numba import cuda


@cuda.jit
def add_kernel_3d(x, y, out):
    i, j, k = cuda.grid(3)
    size = x.shape
    if i < size[0] and j < size[1] and k < size[2]:
        out[i, j, k] = x[i, j, k] + y[i, j, k]

In [76]:
import cupy as cp

n = 500
x = cp.arange(n ** 3).reshape(n, n, n)
y = x * 2
out = cp.empty_like(x)

threads_per_block = (8, 8, 8)
blocks_per_grid = (4, 4, 4)

%timeit -r 1 -n 10 add_kernel_3d[threads_per_block, blocks_per_grid](x, y, out)

6.38 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [61]:
cp.get_default_memory_pool().free_all_blocks()
cp.get_default_pinned_memory_pool().free_all_blocks()